<a href="https://colab.research.google.com/github/mertyyanik/Natural-Language-Processing-Real-or-Fake/blob/master/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doğal Dil İşleme - Gerçek ya da Sahte?
Doğal Dil işleme (Natural Language Processing), basit hali ile yapay zeka ve dil biliminin birleşmesi ile ortaya çıkmış bir kavramdır. Bu kavram herkesin günlük konuşma dilinin bilgisayara öğretilmesi ve bazı çalışmaların bilgisayara yaptırılmasına sebep olmuştur. Günümüze yakın tarihlere baktığımız zaman bilgisayarların güçlenmesini takip eden makine öğrenmesi modelleri daha da gelişmiş ve bir çok alanda kullanılmaya başlanmıştır. Bu alanlardan biriside doğal dil işlemedir. Doğal dil işlemeyi makine öğrenmesi modelleri ile birleştirdiğimiz zaman ortaya bir cümlenin kime ait olduğunu ya da bir cümleyi söyleyen kişinin nasıl bir ruh haline sahip olduğunu anlayabilir noktaya gelmiş bulunmaktayız.

Bizimde amacımız buna benzer bir çalışma yapmaktır. Elimizde haber sitelerinin yazmış olduğu haber başlıkları ve bu başlıkların gerçek ya da sahte olduğunu gösteren bir veri kümesi bulunmaktadır. Derin öğrenme yöntemlerini kullanarak bu veri kümesini bilgisayara öğretecek ve gelecekte yazılmış bir haber başlığının gerçek mi yoksa sahte mi olduğunu anlamaya çalışacağız.

Bunun için Google'ın geliştirmiş olduğu Colab ortamını kullanıyoruz. Bu ortamın avantajı derin öğrenme yöntemlerinde uygulamaları eğitirken **TPU**'ları kullanıyor olmasıdır. TPU'lar, GPU veya CPU'ya göre 15 kat daha hızlıdır. Bu da bizi büyük bir oranda zaman maliyetinden kurtarmaktadır.

Öncelikle elimizde ki veri kümelerini Colab ortamına aktarmamız gerekiyor. Bunu yapmadan önce ortamın Google Drive ile olan bağlantısını onaylamamız lazım. Aşağıda ki kod bloğunu çalıştırdığımız zaman ekranda bir link belirmektedir bu linke tıklayarak karşımıza çıkan kodu output kısmına yapıştırarak Google Drive ile olan bağlantıyı sağlamış oluyoruz.

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
 
from pydrive.drive import GoogleDrive
 
from google.colab import auth
 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
 
gauth = GoogleAuth()
 
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

Bundan sonraki aşama veri kümelerini Google Drive'dan indirmektir. Burada ki önemli nokta dosya numarasının (File ID) elde edilmesidir. Bu numaraya  Google Drive üzerinden veri kümesinin paylaş butonuna basarak ulaşabilirsiniz.

In [0]:
#Test.csv Dosyası
from google.colab import files

file_id = '1Zx7P0t9lQLP4aj4YOUaZDxNRrSkZOTk4'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('test.csv')

#Test.txt Dosyası
from google.colab import files
 
file_id = '1S2-N2HSIgtZpSzPmAEZWmVxV2YlOzCzs'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('test.txt')

#clean_real-Train.txt Dosyası
from google.colab import files
 
file_id = '1pnsf7izWzNYlA41xcWrCXQ5SSFe5JCUF'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_real-Train.txt')

#clean_fake-Train.txt Dosyası
from google.colab import files
 
file_id = '1Mc1m6S6WCJuRkMBWM9_LZGHSzJIw2-e0'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_fake-Train.txt')


Bu noktadan sonra artık kendi Python kodlamanızı istediğiniz gibi yazabiliriz.

Öncelikle gerekli kütüphaneleri yükleyelim.

In [0]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
import time

Text dosyasında bulunan veri dosyalarımızı ayrı ayrı listelere aktarıyoruz. Böylece Python'ın nimetlerinden faydalanabileceğiz.

In [0]:
# Reading Data
input1Data = open('clean_real-Train.txt', 'r+')
input2Data = open('clean_fake-Train.txt', 'r+')
testData = open('test.txt')

testDataForComparison = pd.read_csv('test.csv').iloc[:, 1:]

input1List = []
input2List = []
testList = []

read = input1Data.readline()
read2 = input2Data.readline()
read3 = testData.readline()
rowCount = 0
while read != "":
    input1List.append(read)
    read = input1Data.readline()
    
while read2 != "":
    input2List.append(read2)
    read2 = input2Data.readline()

while read3 != "":
    testList.append(read3)
    read3 = testData.readline()
 

Daha sonra iki farklı veri kümesinde bulunan 'real' ve 'fake' veri kümelerini birleştiriyoruz. Real olanlara 1, fake olanlara 0 yazıyoruz.

In [0]:
# Real and Fake data combined.    
liste = ['real' for i in range(len(input1List))]
realInput = np.column_stack((input1List, liste))


liste = ['fake' for i in range(len(input2List))]
fakeInput = np.column_stack((input2List, liste))

mainInput = realInput.tolist() + fakeInput.tolist()
random.shuffle(mainInput)

sentenceInput = []
targetInput = []
targetInputFloat = []
for i in range(len(mainInput)):
    sentenceInput.append(mainInput[i][0])
    
for i in range(len(mainInput)):
    targetInput.append(mainInput[i][1])
    
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
targetInputFloat = lb.fit_transform(targetInput)

## Veri Ön İşleme
Bundan sonraki aşama veri ön işlemenin başladığı kısım. Veri kümesini bilgisayarın anlayabileceği sayılara çevirmeden önce bakıldığı zaman görülen bazı yanlışlar yok mu? Tabiki var. Bu yanlışların ne olduğundan bahsetmemiz ve en nihayetinde bunlardan kurtulmamız lazım. Yani veri kümemizi bazı veri ön işleme aşamalarından geçirmemiz lazım. Bu veri ön işleme aşamalarını birkaç adımda sıralayabiliriz.
1. Bunlardan ilki noktalama işaretleri. Veri kümesi içerisinde noktalama işaretleri kelimelerin bir parçası olmadığı için veri kümesinden silinmelidir. İleride deyineceğim ama biz temelinde kelimeleri 0 ve daha farklı sayılardan oluşan bir matris haline getirmeye çalışıyoruz. Burada eğer noktalama işaretlerini yok etmezsek bilgisayar bunları da bir sayı olarak düşünür ve yanıltıcı bir durum ortaya çıkar. Bu istediğimiz bir durum değil.
2. İkincisi büyük harfler. İlk harfi büyük olan  Mehmet ile ilk harfi küçük olan mehmet farklı anlamamı gelmeli? Tabiki hayır bu nedenle elimizde ki veri kümesinde büyük harfleri küçük harflere çevirerek bu ayrımı ortadan kaldırmış oluyoruz.
3. Tıpkı büyük harf küçük harf durumunda olduğu gibi Ahmet’in ya da Ahmet’e gibi kelimelerin birbirinden farklılığı olmaması gerekir. Bu nedenle bir kelimenin başına veya sonuna gelen bütün eklerden kurtulmuş olması gerekir. Yani kelimelerin sadece köklerden oluşmasını isteriz.
4. Birleştirici Kelimeler. Bu durumunda diğerlerinden hiçbir farkı yok. ‘ve’, ‘veya’, ‘ile’ gibi bağlaçların silinmesi gerekir. Burada bütün adımlarda yaptığımız temel amaç aynıdır. Bu adımların sonunda geriye sadece kelimelerin kökleri kalmalıdır. Bunun dışında hiçbir şey istemiyoruz. Şimdi gelin bu işlerin Python kodlamasında nasıl yapıldığına bir bakalım.

Burada ilk 3 adım gerekli kütüphanelerin çağrıldığı kısımdır. Bizi asıl ilgilendiren ‘for’ döngüsünün başladığı kısım. Bu döngünün amacı her cümleye tek tek erişmektir. Yani bu döngü her çalıştığında biz bir satırda ki cümleye ulaşıyoruz. Dolayısıyla o cümleye ulaştığımıza göre her şeyi tek bir döngü içerisinde yapabiliriz. İlk aşamada düzenli ifadeler denilen bir kavram ortaya çıkıyor. Bu kavram bilgisayar bilimlerinde çok ayrıntılı bir şekilde anlatılan ve esasında biz bir programlama dili oluştururken o dile kurallar verdiğimiz kısımda kendini gösteriyor. Burada tek bilmemiz gereken şey 2 büyük parantez içerisinde ki ifadede a-z bütün küçük harflerin, A-Z bütün büyük ifadelerin ve tırnak işaretinin bütün küçük harflerin ve bütün büyük harflerin dışında kalan ifadeleri temsil ediyor olmasıdır. İkinci kısımda ki boşluk karakteri, bu ifadelerden herhangi biriyle karşılaşılır ise o ifadeyi boşluk karakteri ile yer değiştirecektir. Bunu yaptığımız zaman bütün noktalama işaretlerinden kurtulmuş oluyoruz.

Hatırlarsanız ikinci adımımız büyük harflerden kurtulmaktı. Burada da ikinci adımda lower() fonksiyonunu kullanarak bütün büyük harfleri küçük harfler ile yer değiştirmiş olduk.
Daha sonrasında yapmamız gereken şey kelime eklerinden ve bağlaç gibi cümleden bağımsız olan kelimelerden kurtulmak. Bunu yapmadan önce elimizde her bir kelimenin olduğu bir liste olması gerekiyor. Split() fonksiyonu cümledeki her bir kelimeyi bir listeye aktarıyor. Şimdi artık bu kelimelerin her birine tek tek ulaşarak o kelimenin üzerinde işlem yapabiliriz.
Bir alt satırda ki kod parçasında bir döngü ile her bir kelimeye tek tek ulaşıyor ve eğer bu kelime bir stopwords değilse o zaman stem() fonksiyonu ile başında veya sonunda bulunan eklerden kurtulmasını sağlıyoruz. Eğer bir stopwords ise o zaman kelimeyi tamamen yok ediyor.
Bütün bu adımların sonunda artık elimizde o 4 adımda istediğimiz sadece köklerden oluşan bir veri kümesi olması gerekiyor. Bakalım gerçekten öyle olmuş mu?





In [8]:
# Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemmer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Regular Expressions
import re

sentenceList = []
for i in range(len(sentenceInput)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', sentenceInput[i]) #-> 1.Adım
    # Upper Case Problem
    comment = comment.lower() #-> 2. Adım
    # Splitting
    comment = comment.split() #-> 3.Adım
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))] # -> 4. Adım
    
    comment = ' '.join(comment)
    sentenceList.append(comment)
    
temporaryResult = pd.DataFrame(data=sentenceList)
print(temporaryResult)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
                                                      0
0                    donald trump begin asia trip japan
1          donald trump call media enemi american peopl
2                   obama tell trump presid famili busi
3     bad jewish loser lament trump victori bbc news...
4     face nobodi care wikileak trump doesnt tempera...
5                   fool trump new messiah includ video
6                                   deep trump truth go
7                 donald trump support fight rig system
8     donald trump hit shadi voter intimid lawsuit a...
9     donald trump plan make america great could cos...
10    dilbert creator endors donald trump simpl reas...
11              trump prais poll said rig point florida
12         trump slam stein recount push scam say elect
13           mcmaster deni trump reveal classifi inform
14                      trump pick loyali

Evet, gerçekten veri kümesi içerisinde sadece köklerin olduğunu ve herhangi bir bağlaç kalmadığını görebiliyoruz. Tam olarak istediğimiz durum buydu. Artık veri kümesini bilgisayarın anlayabileceği 0 ve 1 lerden oluşan bir matris haline getirebiliriz.

Bunun için kullandığımız Python kütüphanesi Scikit-Learn. Bu kütüphanenin içerisindeki CountVectorizer sınıfını kullanarak bu işlemi yapabiliyoruz. Bu sınıf veri kümesinde ki bütün kelimeleri alıp birer değişken haline getiriyor ve her bir cümlede o kelimenin olup olmadığına göre 0 ve 1 sayısını veriyor. En nihayetinde elimizde çok fazla sayıda 0 ve çok az sayıda 1’lerden oluşan bir sparse matris oluşuyor.


In [0]:
# Vectorized All Data
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentenceList)

result = pd.DataFrame(data=X.toarray())

Şu noktadan itibaren bu veri kümesi artık bilgisayarın anlayabileceği bir model haline geldi.

Test veri kümemiz ile eğitim veri kümemiz birbirinden tamamen ayrı. Bu nedenle bu işlemlerin birebir aynısı test veri kümesi içinde uygulanmıştır.


In [0]:
# Test Data
testSentenceList = []
for i in range(len(testList)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', testList[i])
    # Upper Case Problem
    comment = comment.lower()
    # Splitting
    comment = comment.split()
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))]
    
    comment = ' '.join(comment)
    testSentenceList.append(comment)

X = vectorizer.transform(testSentenceList)

testResult = pd.DataFrame(data=X.toarray())
# PCA for Test
#testResult = pca.transform(testResult)


## Derin Öğrenme
Nihayet artık veri kümesi bir model oluşturmaya hazır hale gelmiştir. Modelimiz bir derin öğrenme modelidir. Dolayısıyla derin öğrenmenin ne olduğundan kısaca bahsetmekte fayda var. Derin öğrenme, basit yapay sinir ağlarının daha karmaşık hale getirilmişi ve her bir katmana istediğimiz gibi müdahale edebildiğimiz bir yapısıdır. Karmaşıklık arttıkça derin öğrenmeye olan ihtiyaçta artıyor.


[![image](https://i.hizliresim.com/9a0AGO.jpg)](https://hizliresim.com/9a0AGO)

Şimdi nasıl bir model tasarladığıma kısaca bir bakalım. Elimizde 1 tane giriş katmanı, 1 tane saklı katman ve 2 tane seyreltme katmanı bulunmaktadır. Giriş ve saklı katman 3000’er tane nörona sahiptir. 2 tane seyreltme katmanımız var ve bu katmanlar 0.5 eşik değerine sahip. Bu seyreltme katmanlarının temel amacı ezberlemeyi azaltmaktır. Bu istemediğimiz bir durum. Dolayısıyla bu seyreltme katmanları eşik değerini 0.5 olarak girdiğimiz için burada 1500 tane olan nöron sayısını rasgele olarak yarıya düşürüyor. Bu noktadan sonra sonucu gerçek ya da yalan olarak belirlenecek olan tek bir nörona sahip bir tane çıkış katmanı oluşturuluyor. Daha geniş bir açıdan baktığımız zaman 1 tane giriş katmanımız, 1 tane saklı katmanımız ve 1 tanede çıkış katmanımız oluşuyor. Şimdi bunun nasıl kodlandığına bakalım.


In [0]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import TensorBoard
from keras import optimizers
import datetime

time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

kerasboard = TensorBoard(log_dir="/tmp/tensorboard/{}".format(time))

#result = result.reshape((2777, 1, 2000))
#testResult = testResult.reshape((489, 1, 2000))

classifier = Sequential()

classifier.add(Dense(1500, kernel_initializer="uniform", activation='sigmoid', input_dim=3854)) #1 -> Giriş Katmanı
#classifier.add(LSTM(1500, input_shape=(1, 2000), activation='sigmoid',return_sequences=False))
classifier.add(Dropout(0.50))


classifier.add(Dense(1500,kernel_initializer='uniform' ,activation='sigmoid')) #2 -> Saklı Katman
classifier.add(Dropout(0.50))


classifier.add(Dense(1, kernel_initializer="uniform", activation='sigmoid')) #3 -> Çıkış Katmanı

adam = optimizers.Adam(lr=0.0001, decay=0.00008)

classifier.compile(optimizer=adam,loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(result, targetInputFloat, epochs=40, callbacks=[kerasboard])

prediction = classifier.predict(testResult)

prediction = (prediction > 0.5)

from sklearn.metrics import accuracy_score

acScoreNN = accuracy_score(lb.fit_transform(testDataForComparison), prediction)
print("Accuracy : " + str(acScoreNN))

probaNN = classifier.predict_proba(testResult)
print("NN AUC " + str(metrics.roc_auc_score(lb.fit_transform(testDataForComparison), probaNN)))

print("tensorboard --logdir="+kerasboard.log_dir)

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

#Activate Tensorboard
%tensorboard --logdir=/tmp/tensorboard/2019_03_30_10_34_18

In [0]:
#Saving the best model with Pickle (Neural %84.04)
import pickle
pickle.dump(classifier, open("NeuralNews", 'wb'))

loading = pickle.load(open("NeuralNews", 'rb'))
predictionPickleNeural = loading.predict(testResult)

predictionPickleNeural = (predictionPickleNeural > 0.5)

acScorePickleNeural = accuracy_score(lb.fit_transform(testDataForComparison), predictionPickleNeural)
print("Accuracy Pickle Neural : " + str(acScorePickleNeural))